In [1]:
#import Ipynb_importer
#import DbOpModule
import pymysql
import tushare as ts
import numpy as np
import pandas as pd
import time
import matplotlib.pyplot as plt
import talib as ta
import sys
import datetime
import json
import pymongo
import math
from copy import deepcopy
from pandas.io.json import json_normalize

In [2]:
class MongoDbOp():
    host = "localhost"
    port = 27017
    db_name = None
    db_col = None
    db_op = None
    col_op = None

    def __init__(self, host_ = "localhost", port_ = 27017):
        host = host_
        port = port_

    def connect_To_MongoDb(self, host_ = "localhost", port_ = 27017, db_name = "test_db", col_name = "test_col"):
        self.db_name = db_name
        self.db_col  = col_name

        client = pymongo.MongoClient(host='localhost', port = 27017)
        self.db_op = client[self.db_name]
        self.col_op = self.db_op[self.db_col]


    def insert_stock_daily_many(self, df ):

        for i in df.index:
            df.loc[i,'_id'] = f"{df.loc[i,'ts_code']}-{df.loc[i,'trade_date']}"   # 增加 '_id' 字段， 为ts_code-trade_date, 如 “000001.SZ-20220801"
 
        data = json.loads(df.T.to_json()).values()
        try:
            res = self.col_op.insert_many(data)
        except  Exception as e:
            res = e
            print ("insert_stock_daily_many  Error: " +str(e) )
        
        return res
        


    def update_stock_daily_many(self, df):

        for i in df.index:
            df.loc[i,'_id'] = f"{df.loc[i,'ts_code']}-{df.loc[i,'trade_date']}"   # 增加 '_id' 字段， 为ts_code-trade_date, 如 “000001.SZ-20220801"
 
        data = json.loads(df.to_json(orient="index",force_ascii=False)).values()
        for row in data:
            self.col_op.update_one({"_id": f"{row['ts_code']}-{row['trade_date']}"},{"$set":row}, upsert=True)
            #self.col_op.update_one({"ts_code":row['ts_code'], "trade_date":row["trade_date"]},{"$set":row}, upsert=True)
        


    def find_code(self, code):
        ret = list(self.col_op.find({"ts_code":code}))
        
        if(len(ret) == 0):
            return None
        else:
            df = json_normalize(ret)
            df = df.drop(columns=["_id"])
            return df   
        
    def find_any(self, qurey):
        ret = list(self.col_op.find(qurey))
        
        if(len(ret) == 0):
            return None
        else:
            df = json_normalize(ret)
            #df = df.drop(columns=["_id"])
            return df      


    def del_all_data_in_collection(self, qurey = {}):
        self.col_op.delete_many(qurey)


    def del_collection(self,col_name):
        return self.db_op[col_name].drop()
        

    def del_database(self):
        return self.db_op.dropDatabase()
        




In [3]:
class MongoDbOp2():
    host = "localhost"
    port = 27017
    db_name = None
    col_sh_name = "SH_stock_daily"
    col_sz_name = "SZ_stock_daily"
    col_bj_name = "BJ_stock_daily"

    op_db = None
    op_col = None

    op_col_sh = None
    op_col_sz = None
    op_col_bj = None

    def __init__(self, host_ = "localhost", port_ = 27017):
        self.host = host_
        self.port = port_

    def connect_To_MongoDb(self, host_ = "localhost", port_ = 27017, db_name = "test_db"):
        self.db_name = db_name
        self.host = host_
        self.port = port_

        client = pymongo.MongoClient(self.host, self.port)
        self.op_db = client[self.db_name]
        self.op_col_sh = self.op_db[self.col_sh_name]
        self.op_col_sz = self.op_db[self.col_sz_name]
        self.op_col_bj = self.op_db[self.col_bj_name]

    


    def insert_stock_daily_many(self, df ):

        for i in df.index:
            df.loc[i,'_id'] = f"{df.loc[i,'ts_code']}-{df.loc[i,'trade_date']}"   # 增加 '_id' 字段， 为ts_code-trade_date, 如 “000001.SZ-20220801"
 
        data = json.loads(df.T.to_json()).values()
        try:
            res = self.col_op.insert_many(data)
        except  Exception as e:
            res = e
            print ("insert_stock_daily_many  Error: " +str(e) )
        
        return res
        


    def update_stock_daily_many(self, df):

        for i in df.index:
            df.loc[i,'_id'] = f"{df.loc[i,'ts_code']}-{df.loc[i,'trade_date']}"   # 增加 '_id' 字段， 为ts_code-trade_date, 如 “000001.SZ-20220801"
 
        data = json.loads(df.to_json(orient="index",force_ascii=False)).values()
        for row in data:
            self.col_op.update_one({"_id": f"{row['ts_code']}-{row['trade_date']}"},{"$set":row}, upsert=True)
            #self.col_op.update_one({"ts_code":row['ts_code'], "trade_date":row["trade_date"]},{"$set":row}, upsert=True)
        


    def find_code(self, code):
        ret = list(self.col_op.find({"ts_code":code}))
        
        if(len(ret) == 0):
            return None
        else:
            df = pd.json_normalize(ret)
            df = df.drop(columns=["_id"])
            return df   
        
    def find_any(self, qurey):
        ret = list(self.col_op.find(qurey))
        
        if(len(ret) == 0):
            return None
        else:
            df = pd.json_normalize(ret)
            #df = df.drop(columns=["_id"])
            return df      


    def del_all_data_in_collection(self, qurey = {}):
        self.col_op.delete_many(qurey)


    def del_collection(self,col_name):
        return self.db_op[col_name].drop()
        

    def del_database(self):
        return self.db_op.dropDatabase()
        




In [4]:
class MongoDbOp_org():
    host = "localhost"
    port = 27017
    db_name = None

    op_db = None
    op_col = None

    def __init__(self, host_ = "localhost", port_ = 27017):
        self.host = host_
        self.port = port_

    def connect_To_MongoDb(self, host_ = "localhost", port_ = 27017, db_name = "test_db"):
        self.db_name = db_name
        self.host = host_
        self.port = port_

        client = pymongo.MongoClient(self.host, self.port)
        self.op_db = client[self.db_name]


    def insert_stock_daily_many(self, col_name, df ):
        # 把 “ts_code "字段替换为 ”stock_code"
        for colume in df.columns:
            if (colume == 'ts_code'):
                df.rename(columns = {"ts_code":"stock_code"},inplace=True)

        self.op_col = self.op_db[col_name]
        for i in df.index:
            df.loc[i,'_id'] = f"{df.loc[i,'stock_code']}-{df.loc[i,'trade_date']}"   # 增加 '_id' 字段， 为ts_code-trade_date, 如 “000001.SZ-20220801"
 
        data = json.loads(df.T.to_json()).values()
        try:
            res = self.op_col.insert_many(data)
        except  Exception as e:
            res = e
            print ("insert_stock_daily_many  Error: " +str(e) )
        
        return res
        
    def insert_stock_daily_many(self, col_name, df ):
        # 把 “ts_code "字段替换为 ”stock_code"
        for colume in df.columns:
            if (colume == 'ts_code'):
                df.rename(columns = {"ts_code":"stock_code"},inplace=True)

        self.op_col = self.op_db[col_name]
        for i in df.index:
            df.loc[i,'_id'] = f"{df.loc[i,'stock_code']}-{df.loc[i,'trade_date']}"   # 增加 '_id' 字段， 为ts_code-trade_date, 如 “000001.SZ-20220801"
 
        data = json.loads(df.T.to_json()).values()
        try:
            res = self.op_col.insert_many(data)
        except  Exception as e:
            res = e
            print ("insert_stock_daily_many  Error: " +str(e) )
        
        return res

    def insert_stock_daily_many_2(self, col_name, df, step=20):
        
        # 把 “ts_code "字段替换为 ”stock_code"
        for colume in df.columns:
            if (colume == 'ts_code'):
                df.rename(columns = {"ts_code":"stock_code"},inplace=True)


        self.op_col = self.op_db[col_name]

        for i in range(math.ceil(df.shape[0] / step)):
            data = df.iloc[step * i:step * (i + 1)].to_dict(orient="record")
            self.op_col.insert_many(data)


    def update_stock_daily_many(self,col_name, df):
        # 把 “ts_code "字段替换为 ”stock_code"
        for colume in df.columns:
            if (colume == 'ts_code'):
                df.rename(columns = {"ts_code":"stock_code"},inplace=True)

        self.op_col = self.op_db[col_name]

         # 增加 '_id' 字段， 为stock_code-trade_date, 如 “000001.SZ-20220801"
        for i in df.index:
            df.loc[i,'_id'] = f"{df.loc[i,'stock_code']}-{df.loc[i,'trade_date']}"  

        data = json.loads(df.to_json(orient="index",force_ascii=False)).values()
        for row in data:
            self.op_col.update_one({"_id": f"{row['stock_code']}-{row['trade_date']}"},{"$set":row}, upsert=True)
            #self.col_op.update_one({"ts_code":row['ts_code'], "trade_date":row["trade_date"]},{"$set":row}, upsert=True)
        
    def update_stock_basic_info_many(self,col_name, df):
        # 把 “ts_code "字段替换为 ”stock_code"
        for colume in df.columns:
            if (colume == 'ts_code'):
                df.rename(columns = {"ts_code":"stock_code"},inplace=True)

        self.op_col = self.op_db[col_name]

        data = json.loads(df.to_json(orient="index",force_ascii=False)).values()
        for row in data:
            self.op_col.update_one({"_id": row['stock_code']},{"$set":row}, upsert=True)
            #self.op_col.update_one({"_id": f"{row['ts_code']}-{row['trade_date']}"},{"$set":row}, upsert=True)
            #self.col_op.update_one({"ts_code":row['ts_code'], "trade_date":row["trade_date"]},{"$set":row}, upsert=True)
        
    def update_stock_divid_factors(self,col_name, disc_):

        self.op_col = self.op_db[col_name]
        disc = deepcopy(disc_)

        self.op_col.update_one({'_id': disc["_id"]},{"$set":disc}, upsert=True)


    def update_stock_divid_factors_many(self,col_name, disc_list):

        self.op_col = self.op_db[col_name]

        self.op_col.insert_many(disc_list)


    def find_code(self,col_name, code):

        self.op_col = self.op_db[col_name]
        ret = list(self.op_col.find({"stock_code":code}))
        
        if(len(ret) == 0):
            return None
        else:
            df = json_normalize(ret)
            df = df.drop(columns=["_id"])
            return df   
        
    def find_any(self, col_name, qurey):
        self.op_col = self.op_db[col_name]
        ret = list(self.op_col.find(qurey))
        
        if(len(ret) == 0):
            return None
        else:
            df = json_normalize(ret)
            #df = df.drop(columns=["_id"])
            return df      


    def del_all_data_in_collection(self,col_name, qurey = {}):
        self.op_col = self.op_db[col_name]
        self.op_col.delete_many(qurey)


    def del_collection(self,col_name):
        return self.op_col[col_name].drop()
        

    def del_database(self):
        return self.op_db.dropDatabase()
        




In [5]:
class MongoDbOp2():
    host = "localhost"
    port = 27017
    db_name = None
    col_sh_name = "SH_stock_daily"
    col_sz_name = "SZ_stock_daily"
    col_bj_name = "BJ_stock_daily"

    op_db = None
    op_col = None

    op_col_sh = None
    op_col_sz = None
    op_col_bj = None

    def __init__(self, host_ = "localhost", port_ = 27017):
        self.host = host_
        self.port = port_

    def connect_To_MongoDb(self, host_ = "localhost", port_ = 27017, db_name = "test_db"):
        self.db_name = db_name
        self.host = host_
        self.port = port_

        client = pymongo.MongoClient(self.host, self.port)
        self.op_db = client[self.db_name]
        self.op_col_sh = self.op_db[self.col_sh_name]
        self.op_col_sz = self.op_db[self.col_sz_name]
        self.op_col_bj = self.op_db[self.col_bj_name]

    


    def insert_stock_daily_many(self, df ):

        for i in df.index:
            df.loc[i,'_id'] = f"{df.loc[i,'ts_code']}-{df.loc[i,'trade_date']}"   # 增加 '_id' 字段， 为ts_code-trade_date, 如 “000001.SZ-20220801"
 
        data = json.loads(df.T.to_json()).values()
        try:
            res = self.col_op.insert_many(data)
        except  Exception as e:
            res = e
            print ("insert_stock_daily_many  Error: " +str(e) )
        
        return res
        


    def update_stock_daily_many(self, df):

        for i in df.index:
            df.loc[i,'_id'] = f"{df.loc[i,'ts_code']}-{df.loc[i,'trade_date']}"   # 增加 '_id' 字段， 为ts_code-trade_date, 如 “000001.SZ-20220801"
 
        data = json.loads(df.to_json(orient="index",force_ascii=False)).values()
        for row in data:
            self.col_op.update_one({"_id": f"{row['ts_code']}-{row['trade_date']}"},{"$set":row}, upsert=True)
            #self.col_op.update_one({"ts_code":row['ts_code'], "trade_date":row["trade_date"]},{"$set":row}, upsert=True)
        


    def find_code(self, code):
        ret = list(self.col_op.find({"ts_code":code}))
        
        if(len(ret) == 0):
            return None
        else:
            df = pd.json_normalize(ret)
            df = df.drop(columns=["_id"])
            return df   
        
    def find_any(self, qurey):
        ret = list(self.col_op.find(qurey))
        
        if(len(ret) == 0):
            return None
        else:
            df = pd.json_normalize(ret)
            #df = df.drop(columns=["_id"])
            return df      


    def del_all_data_in_collection(self, qurey = {}):
        self.col_op.delete_many(qurey)


    def del_collection(self,col_name):
        return self.db_op[col_name].drop()
        

    def del_database(self):
        return self.db_op.dropDatabase()
        




In [6]:
pro = ts.pro_api('af5f423bb57f376e71c2903058362f8e3e053fe30f486463f9bdac8e')
#df = ts.pro_bar(ts_code="000001.SZ", adj='qfq', start_date= '20220701' , end_date='20220805')
df = pro.stock_basic(exchange='', list_status='L', fields='ts_code,symbol,name,area,industry,market,list_date')

In [7]:
list(df["ts_code"])


['000001.SZ',
 '000002.SZ',
 '000004.SZ',
 '000005.SZ',
 '000006.SZ',
 '000007.SZ',
 '000008.SZ',
 '000009.SZ',
 '000010.SZ',
 '000011.SZ',
 '000012.SZ',
 '000014.SZ',
 '000016.SZ',
 '000017.SZ',
 '000019.SZ',
 '000020.SZ',
 '000021.SZ',
 '000023.SZ',
 '000025.SZ',
 '000026.SZ',
 '000027.SZ',
 '000028.SZ',
 '000029.SZ',
 '000030.SZ',
 '000031.SZ',
 '000032.SZ',
 '000034.SZ',
 '000035.SZ',
 '000036.SZ',
 '000037.SZ',
 '000038.SZ',
 '000039.SZ',
 '000040.SZ',
 '000042.SZ',
 '000045.SZ',
 '000046.SZ',
 '000048.SZ',
 '000049.SZ',
 '000050.SZ',
 '000055.SZ',
 '000056.SZ',
 '000058.SZ',
 '000059.SZ',
 '000060.SZ',
 '000061.SZ',
 '000062.SZ',
 '000063.SZ',
 '000065.SZ',
 '000066.SZ',
 '000068.SZ',
 '000069.SZ',
 '000070.SZ',
 '000078.SZ',
 '000088.SZ',
 '000089.SZ',
 '000090.SZ',
 '000096.SZ',
 '000099.SZ',
 '000100.SZ',
 '000150.SZ',
 '000151.SZ',
 '000153.SZ',
 '000155.SZ',
 '000156.SZ',
 '000157.SZ',
 '000158.SZ',
 '000159.SZ',
 '000166.SZ',
 '000301.SZ',
 '000333.SZ',
 '000338.SZ',
 '0004

In [8]:
db_name = "stock"
col_name = "stock_daily"
stock_basic_col_name = "stock_basic_info"
stock_divid_factors_col_name = "stock_divid_factors"

db_op = MongoDbOp_org()
db_op.connect_To_MongoDb(db_name= db_name)

In [9]:
db_op.update_stock_basic_info_many(col_name= stock_basic_col_name , df = df)
ret = db_op.find_any(stock_basic_col_name,{})
ret

,_id,area,industry,list_date,market,name,stock_code,symbol
0,000001.SZ,深圳,银行,19910403,主板,平安银行,000001.SZ,000001
1,000002.SZ,深圳,全国地产,19910129,主板,万科A,000002.SZ,000002
2,000004.SZ,深圳,软件服务,19910114,主板,ST国华,000004.SZ,000004
3,000005.SZ,深圳,环境保护,19901210,主板,ST星源,000005.SZ,000005
4,000006.SZ,深圳,区域地产,19920427,主板,深振业A,000006.SZ,000006
...,...,...,...,...,...,...,...,...
4856,689009.SH,北京,摩托车,20201029,CDR,九号公司-WD,689009.SH,689009
4857,301132.SZ,江西,元器件,20220810,创业板,N满坤,301132.SZ,301132
4858,301192.SZ,None,None,20220811,创业板,泰祥股份,301192.SZ,301192
4859,301318.SZ,深圳,通信设备,20220810,创业板,N维海德,301318.SZ,301318


In [10]:
#db_op.insert_stock_daily_many(col_name, df)


In [11]:
#db_op.insert_stock_daily_many_2(col_name,df, 15)

In [12]:
#time.sleep(1)
#db_op.del_all_data_in_collection(stock_basic_col_name)
#db_op.del_all_data_in_collection(stock_divid_factors_col_name)
#time.sleep(1)

In [13]:
#db_op.update_stock_daily_many(col_name,df)

In [14]:
time.sleep(1)
#db_op.del_all_data_in_collection()
time.sleep(1)

In [15]:
#val_ret = db_op.find_code("002920.SZ")
#val_ret = db_op.find_any(col_name,{})
val_ret = db_op.find_any(stock_divid_factors_col_name,{})
val_ret


,_id,updateTime,value
0,000001.SZ,20220810,"{682358400000: [1.0, 0.0, 0.0, 0.0, 0.0, 0, 1...."
1,000002.SZ,20220810,"{675705600000: [0.0, 0.0, 0.0, 0.5, 4.4, 0, 1...."
2,000004.SZ,20220810,"{678038400000: [0.0, 0.2, 0.0, 0.0, 0.0, 0, 1...."
3,000005.SZ,20220810,"{780249600000: [0.0, 0.3333, 0.0, 0.0, 0.0, 0,..."
4,000006.SZ,20220810,"{739382400000: [0.0, 0.0, 0.0, 0.3, 12.8, 0, 1..."
...,...,...,...
4852,871981.BJ,20220810,{}
4853,872925.BJ,20220810,{}
4854,873169.BJ,20220810,{}
4855,873223.BJ,20220810,{}


In [16]:
df3 = pro.stock_basic(exchange='', list_status='L', fields='ts_code,symbol,name,area,industry,market,list_date')
df3

,ts_code,symbol,name,area,industry,market,list_date
0,000001.SZ,000001,平安银行,深圳,银行,主板,19910403
1,000002.SZ,000002,万科A,深圳,全国地产,主板,19910129
2,000004.SZ,000004,ST国华,深圳,软件服务,主板,19910114
3,000005.SZ,000005,ST星源,深圳,环境保护,主板,19901210
4,000006.SZ,000006,深振业A,深圳,区域地产,主板,19920427
...,...,...,...,...,...,...,...
4856,871981.BJ,871981,晶赛科技,None,None,北交所,20211115
4857,872925.BJ,872925,锦好医疗,None,None,北交所,20211025
4858,873169.BJ,873169,七丰精工,None,None,北交所,20220415
4859,873223.BJ,873223,荣亿精密,None,None,北交所,20220609


In [17]:

#db_op.update_stock_basic_info_many(col_name= stock_basic_col_name , df = df3)

In [18]:
val = db_op.find_any(stock_basic_col_name ,{})
val

,_id,area,industry,list_date,market,name,stock_code,symbol
0,000001.SZ,深圳,银行,19910403,主板,平安银行,000001.SZ,000001
1,000002.SZ,深圳,全国地产,19910129,主板,万科A,000002.SZ,000002
2,000004.SZ,深圳,软件服务,19910114,主板,ST国华,000004.SZ,000004
3,000005.SZ,深圳,环境保护,19901210,主板,ST星源,000005.SZ,000005
4,000006.SZ,深圳,区域地产,19920427,主板,深振业A,000006.SZ,000006
...,...,...,...,...,...,...,...,...
4856,689009.SH,北京,摩托车,20201029,CDR,九号公司-WD,689009.SH,689009
4857,301132.SZ,江西,元器件,20220810,创业板,N满坤,301132.SZ,301132
4858,301192.SZ,None,None,20220811,创业板,泰祥股份,301192.SZ,301192
4859,301318.SZ,深圳,通信设备,20220810,创业板,N维海德,301318.SZ,301318


In [19]:
df.T

,0,1,2,3,4,5,6,7,8,9,...,4851,4852,4853,4854,4855,4856,4857,4858,4859,4860
stock_code,000001.SZ,000002.SZ,000004.SZ,000005.SZ,000006.SZ,000007.SZ,000008.SZ,000009.SZ,000010.SZ,000011.SZ,...,871396.BJ,871553.BJ,871642.BJ,871857.BJ,871970.BJ,871981.BJ,872925.BJ,873169.BJ,873223.BJ,689009.SH
symbol,000001,000002,000004,000005,000006,000007,000008,000009,000010,000011,...,871396,871553,871642,871857,871970,871981,872925,873169,873223,689009
name,平安银行,万科A,ST国华,ST星源,深振业A,全新好,神州高铁,中国宝安,美丽生态,深物业A,...,常辅股份,凯腾精工,通易航天,泓禧科技,大禹生物,晶赛科技,锦好医疗,七丰精工,荣亿精密,九号公司-WD
area,深圳,深圳,深圳,深圳,深圳,深圳,北京,深圳,深圳,深圳,...,None,None,None,None,None,None,None,None,None,北京
industry,银行,全国地产,软件服务,环境保护,区域地产,酒店餐饮,运输设备,电气设备,建筑工程,区域地产,...,None,None,None,None,None,None,None,None,None,摩托车
market,主板,主板,主板,主板,主板,主板,主板,主板,主板,主板,...,北交所,北交所,北交所,北交所,北交所,北交所,北交所,北交所,北交所,CDR
list_date,19910403,19910129,19910114,19901210,19920427,19920413,19920507,19910625,19951027,19920330,...,20201118,20210806,20210816,20220228,20220518,20211115,20211025,20220415,20220609,20201029


In [20]:
df.T.to_json()

'{"0":{"stock_code":"000001.SZ","symbol":"000001","name":"\\u5e73\\u5b89\\u94f6\\u884c","area":"\\u6df1\\u5733","industry":"\\u94f6\\u884c","market":"\\u4e3b\\u677f","list_date":"19910403"},"1":{"stock_code":"000002.SZ","symbol":"000002","name":"\\u4e07\\u79d1A","area":"\\u6df1\\u5733","industry":"\\u5168\\u56fd\\u5730\\u4ea7","market":"\\u4e3b\\u677f","list_date":"19910129"},"2":{"stock_code":"000004.SZ","symbol":"000004","name":"ST\\u56fd\\u534e","area":"\\u6df1\\u5733","industry":"\\u8f6f\\u4ef6\\u670d\\u52a1","market":"\\u4e3b\\u677f","list_date":"19910114"},"3":{"stock_code":"000005.SZ","symbol":"000005","name":"ST\\u661f\\u6e90","area":"\\u6df1\\u5733","industry":"\\u73af\\u5883\\u4fdd\\u62a4","market":"\\u4e3b\\u677f","list_date":"19901210"},"4":{"stock_code":"000006.SZ","symbol":"000006","name":"\\u6df1\\u632f\\u4e1aA","area":"\\u6df1\\u5733","industry":"\\u533a\\u57df\\u5730\\u4ea7","market":"\\u4e3b\\u677f","list_date":"19920427"},"5":{"stock_code":"000007.SZ","symbol":"00000

In [21]:
js =df.to_json(orient="index",force_ascii=False)
js

'{"0":{"stock_code":"000001.SZ","symbol":"000001","name":"平安银行","area":"深圳","industry":"银行","market":"主板","list_date":"19910403"},"1":{"stock_code":"000002.SZ","symbol":"000002","name":"万科A","area":"深圳","industry":"全国地产","market":"主板","list_date":"19910129"},"2":{"stock_code":"000004.SZ","symbol":"000004","name":"ST国华","area":"深圳","industry":"软件服务","market":"主板","list_date":"19910114"},"3":{"stock_code":"000005.SZ","symbol":"000005","name":"ST星源","area":"深圳","industry":"环境保护","market":"主板","list_date":"19901210"},"4":{"stock_code":"000006.SZ","symbol":"000006","name":"深振业A","area":"深圳","industry":"区域地产","market":"主板","list_date":"19920427"},"5":{"stock_code":"000007.SZ","symbol":"000007","name":"全新好","area":"深圳","industry":"酒店餐饮","market":"主板","list_date":"19920413"},"6":{"stock_code":"000008.SZ","symbol":"000008","name":"神州高铁","area":"北京","industry":"运输设备","market":"主板","list_date":"19920507"},"7":{"stock_code":"000009.SZ","symbol":"000009","name":"中国宝安","area":"深圳","industry":"电气设备",

In [22]:
data = json.loads(js)
data.get("0").get("trade_date")

In [23]:
data = json.loads(df.T.to_json()).values()
data

dict_values([{'stock_code': '000001.SZ', 'symbol': '000001', 'name': '平安银行', 'area': '深圳', 'industry': '银行', 'market': '主板', 'list_date': '19910403'}, {'stock_code': '000002.SZ', 'symbol': '000002', 'name': '万科A', 'area': '深圳', 'industry': '全国地产', 'market': '主板', 'list_date': '19910129'}, {'stock_code': '000004.SZ', 'symbol': '000004', 'name': 'ST国华', 'area': '深圳', 'industry': '软件服务', 'market': '主板', 'list_date': '19910114'}, {'stock_code': '000005.SZ', 'symbol': '000005', 'name': 'ST星源', 'area': '深圳', 'industry': '环境保护', 'market': '主板', 'list_date': '19901210'}, {'stock_code': '000006.SZ', 'symbol': '000006', 'name': '深振业A', 'area': '深圳', 'industry': '区域地产', 'market': '主板', 'list_date': '19920427'}, {'stock_code': '000007.SZ', 'symbol': '000007', 'name': '全新好', 'area': '深圳', 'industry': '酒店餐饮', 'market': '主板', 'list_date': '19920413'}, {'stock_code': '000008.SZ', 'symbol': '000008', 'name': '神州高铁', 'area': '北京', 'industry': '运输设备', 'market': '主板', 'list_date': '19920507'}, {'stock_cod

In [24]:
df = pd.json_normalize(data)
df

,stock_code,symbol,name,area,industry,market,list_date
0,000001.SZ,000001,平安银行,深圳,银行,主板,19910403
1,000002.SZ,000002,万科A,深圳,全国地产,主板,19910129
2,000004.SZ,000004,ST国华,深圳,软件服务,主板,19910114
3,000005.SZ,000005,ST星源,深圳,环境保护,主板,19901210
4,000006.SZ,000006,深振业A,深圳,区域地产,主板,19920427
...,...,...,...,...,...,...,...
4856,871981.BJ,871981,晶赛科技,None,None,北交所,20211115
4857,872925.BJ,872925,锦好医疗,None,None,北交所,20211025
4858,873169.BJ,873169,七丰精工,None,None,北交所,20220415
4859,873223.BJ,873223,荣亿精密,None,None,北交所,20220609


In [25]:
df2= df.loc[0:5,:]
data2 = df2.to_json(orient="index",force_ascii=False)
data2 =json.loads(data2).values()
data2

dict_values([{'stock_code': '000001.SZ', 'symbol': '000001', 'name': '平安银行', 'area': '深圳', 'industry': '银行', 'market': '主板', 'list_date': '19910403'}, {'stock_code': '000002.SZ', 'symbol': '000002', 'name': '万科A', 'area': '深圳', 'industry': '全国地产', 'market': '主板', 'list_date': '19910129'}, {'stock_code': '000004.SZ', 'symbol': '000004', 'name': 'ST国华', 'area': '深圳', 'industry': '软件服务', 'market': '主板', 'list_date': '19910114'}, {'stock_code': '000005.SZ', 'symbol': '000005', 'name': 'ST星源', 'area': '深圳', 'industry': '环境保护', 'market': '主板', 'list_date': '19901210'}, {'stock_code': '000006.SZ', 'symbol': '000006', 'name': '深振业A', 'area': '深圳', 'industry': '区域地产', 'market': '主板', 'list_date': '19920427'}, {'stock_code': '000007.SZ', 'symbol': '000007', 'name': '全新好', 'area': '深圳', 'industry': '酒店餐饮', 'market': '主板', 'list_date': '19920413'}])

client = pymongo.MongoClient(host='localhost', port = 27017)

stock_db = client["stock"]

daily = stock_db["daily"]


type(data)

In [26]:
#daily.update_one(update=df2, upsert =True)
#for i in data2:
    #daily.insert_one(i)

In [27]:
#daily.insert_many(data)

res =  daily.find({"ts_code":"002920.SZ"})

ret = list(res)
ret

df = pd.json_normalize(ret)
df = df.drop(columns=["_id"])
df